In [4]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='b0fc2ae7-9e66-4cb2-b8d2-a822c4c9e74a', project_access_token='p-75e061d9499123c453a6ca3dc6f9faa2a4e23038')
pc = project.project_context


In [6]:
!pip install nltk
!pip install ibm_watson
!pip install cloudant==2.3.1
!pip install watson-developer-cloud==1.5
!pip install --upgrade pixiedust

Requirement already up-to-date: pixiedust in /opt/conda/envs/Python36/lib/python3.6/site-packages (1.1.18)


In [7]:

# @hidden_cell
# The following code contains the credentials for a file in your IBM Cloud Object Storage.
# You might want to remove those credentials before you share your notebook.
credentials_2 = {
    'IAM_SERVICE_ID': 'iam-ServiceId-ce4a4a44-022d-40fe-be9b-e16e72b4153c',
    'IBM_API_KEY_ID': 'w3-S42mlOXATWHVO_5uU5NiQ2R1EiZxELIupujtqVLIa',
    'ENDPOINT': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'IBM_AUTH_ENDPOINT': 'https://iam.cloud.ibm.com/oidc/token',
    'BUCKET': 'bdaproject-donotdelete-pr-ku47cnvx97em5d',
    'FILE': 'data.csv'
}


In [9]:

import json
import watson_developer_cloud
from watson_developer_cloud import NaturalLanguageUnderstandingV1
from watson_developer_cloud.natural_language_understanding_v1 \
  import Features, EntitiesOptions, KeywordsOptions
    
import ibm_boto3
from botocore.client import Config

import re
import nltk
from nltk import word_tokenize,sent_tokenize,ne_chunk

cos = ibm_boto3.client('s3',
                    ibm_api_key_id=credentials_2['IBM_API_KEY_ID'],
                    ibm_service_instance_id=credentials_2['IAM_SERVICE_ID'],
                    ibm_auth_endpoint=credentials_2['IBM_AUTH_ENDPOINT'],
                    config=Config(signature_version='oauth'),
                    endpoint_url=credentials_2['ENDPOINT'])

def get_file(filename):
    '''Retrieve file from Cloud Object Storage'''
    fileobject = cos.get_object(Bucket=credentials_2['BUCKET'], Key=filename)['Body']
    return fileobject

def load_string(fileobject):
    '''Load the file contents into a Python string'''
    text = fileobject.read()
    text = text.strip(b"0xc7")
    return text.decode('latin-1')

def put_file(filename, filecontents):
    '''Write file to Cloud Object Storage'''
    resp = cos.put_object(Bucket=credentials_2['BUCKET'], Key=filename, Body=filecontents)
    return resp

In [10]:
sampleTextFileName = "data.csv"
text = load_string(get_file(sampleTextFileName))
text = text.strip("|'|\"|,|-|_|.|""|<|>|/|.|(|)|")
now_text = text.split('\n')


77974


In [ ]:
import nltk
tokenizer = nltk.RegexpTokenizer(r"\w+")
import json
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.natural_language_understanding_v1 import Features, CategoriesOptions,KeywordsOptions,SentimentOptions
import pandas as pd

authenticator = IAMAuthenticator('Sna9bingjNn5_WehY2FD3VNeRQGUjnPIDTqY4t6NpB4X')
natural_language_understanding = NaturalLanguageUnderstandingV1(
    version='2019-07-12',
    authenticator=authenticator
)
i=1
print(len(now_text))
responses=[]
lst = []
while i< len(now_text)-1:
    now_text1 = now_text[i]
    now_text1 = now_text1.strip("|'|<|>|/|.")
    i+=1
    natural_language_understanding.set_service_url('https://api.us-south.natural-language-understanding.watson.cloud.ibm.com/instances/a79b8129-b17d-4fa8-8703-ca4ee7d3df6e')
    try:
        s = tokenizer.tokenize(now_text1.replace("NEWLINE_TOKEN",'').replace("_"," ").replace("1"," ").replace("2"," ").replace("3"," ").replace("4"," ").replace("5"," ").replace("6"," ").replace("7"," ").replace("8"," ").replace("9"," ").replace("0"," ").lower())
        listToStr = ' '.join([str(elem) for elem in s])
        response = natural_language_understanding.analyze(
            text=listToStr,
            features=Features(sentiment=SentimentOptions(targets=[listToStr]))).get_result()
        lst.append([now_text1.split(',')[0],response.get('sentiment').get('document').get('label'),response.get('sentiment').get('document').get('score')])
        df = pd.DataFrame(lst, columns=['rev_id','sentiment','sentiment_score'])
        project.save_data(file_name = "label_Score_data.csv",data = df.to_csv(index=False),overwrite=True)
    except Exception as e:
        print(e)
        continue

77974
